In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
file_path = "../data/analysis_data/body.h5"
with h5py.File(file_path, "r") as f:
    ts = []
    system_ts = []
    ts_grp = f["ts"]
    for device_key in ts_grp.keys():
        device_grp = ts_grp[device_key]
        ts.append(device_grp["ts"][:].astype(np.int64))
        system_ts.append(device_grp["system_ts"][:].astype(np.int64))

In [ ]:
system_ts0, ts0 = system_ts[0], ts[0]
system_ts1, ts1 = system_ts[1], ts[1]

idx1 = np.searchsorted(system_ts1, system_ts0)
valid_mask = (idx1 > 0) & (idx1 < len(system_ts1))
idx0 = np.where(valid_mask)[0]
idx1 = idx1[valid_mask]

dist_right = np.abs(system_ts1[idx1] - system_ts0[idx0])
dist_left = np.abs(system_ts1[idx1 - 1] - system_ts0[idx0])
closer_to_left = dist_left < dist_right
idx1[closer_to_left] -= 1

time_s = (system_ts0[idx0]-system_ts0[0]) / 1e9
ts_delta_ms = (ts0[idx0]-ts1[idx1]) / 1e3
system_ts_delta_ms = (system_ts0[idx0]-system_ts1[idx1]) / 1e6

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()

ax.plot(time_s, ts_delta_ms)
ax.set_xlabel("time (s)")
ax.set_ylabel("ts delta (ms)")
ax.set_title("Devices clock drift")

plt.grid(True)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()

ax.plot(time_s, system_ts_delta_ms)
ax.set_xlabel("time (s)")
ax.set_ylabel("system ts delta (ms)")
ax.set_title("Capture time alignment")

plt.grid(True)
plt.show()

In [ ]:
median_delta = np.median(ts_delta_ms)
std_delta = np.std(ts_delta_ms)

filter_mask = np.abs(ts_delta_ms-median_delta) < (3*std_delta)

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(time_s[filter_mask], ts_delta_ms[filter_mask])
ax.set_xlabel("time (s)")
ax.set_ylabel("ts delta (ms)")
ax.set_title("Devices clock drift (spikes removed)")

plt.grid(True)
plt.show()

print(f"Original data points: {len(ts_delta_ms)}")
print(f"Spikes removed: {len(ts_delta_ms)-np.sum(filter_mask)}")